In [ ]:
# Install required packages
!pip install transformers huggingface_hub
!pip install PyPDF2

# Import necessary libraries
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import PyPDF2
import io
import os
from google.colab import files

# Authenticate with Hugging Face using your API key
hf_access_token = ""  # Your token here
login(hf_access_token)

# Load the model and tokenizer
model_name = "google/gemma-2b"  # Gemma 2 model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Function to extract text from a PDF file
def extract_text_from_pdf(uploaded_pdf):
    pdf_reader = PyPDF2.PdfReader(uploaded_pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Upload PDF file and extract text
def upload_and_extract_pdf():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Uploaded file: {filename}')
        with open(filename, "rb") as f:
            extracted_text = extract_text_from_pdf(f)
        os.remove(filename)  # Remove the file after extraction to avoid clutter
        return extracted_text

# Summarization function
def summarize_text(text):
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors="pt", truncation=True, padding=True)

    # Define a higher value for max_new_tokens to generate a reasonable summary
    summary = generator(text,
                        max_new_tokens=100,  # Control the number of tokens for the summary
                        num_return_sequences=1,
                        do_sample=False,
                        truncation=True)  # Truncate if the text is too long

    return summary[0]['generated_text']


# Input or PDF upload option
def get_user_input():
    print("Enter your text below or upload a PDF to summarize.")

    # Text input for direct user entry
    text_input = input("Paste your meeting transcript or text (or press Enter to upload a PDF): ")

    if text_input:
        print("Generating summary for your input text...")
        summary = summarize_text(text_input)
    else:
        print("Please upload a PDF file for summarization.")
        extracted_text = upload_and_extract_pdf()
        print("Generating summary for extracted text from PDF...")
        summary = summarize_text(extracted_text)

    print("\nSummary:")
    print(summary)

# Run the user input function
get_user_input()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Enter your text below or upload a PDF to summarize.
Paste your meeting transcript or text (or press Enter to upload a PDF): Gemma, a new family of lightweight, state-of-the-art open models, draws inspiration from Google’s Gemini models and is developed by Google DeepMind and other teams. Available worldwide, Gemma comes in two sizes: Gemma 2B and Gemma 7B, each with pre-trained and instruction-tuned variants. Alongside model weights, a Responsible Generative AI Toolkit aids in creating safer AI applications. Toolchains for inference and supervised fine-tuning are provided for JAX, PyTorch, and TensorFlow, with integration into popular platforms like Hugging Face and NVIDIA NeMo. Gemma models offer easy deployment on various platforms, including Google Cloud, with optimization for industry-leading performance across hardware platforms like NVIDIA GPUs and Google Cloud TPUs.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generating summary for your input text...

Summary:
Gemma, a new family of lightweight, state-of-the-art open models, draws inspiration from Google’s Gemini models and is developed by Google DeepMind and other teams. Available worldwide, Gemma comes in two sizes: Gemma 2B and Gemma 7B, each with pre-trained and instruction-tuned variants. Alongside model weights, a Responsible Generative AI Toolkit aids in creating safer AI applications. Toolchains for inference and supervised fine-tuning are provided for JAX, PyTorch, and TensorFlow, with integration into popular platforms like Hugging Face and NVIDIA NeMo. Gemma models offer easy deployment on various platforms, including Google Cloud, with optimization for industry-leading performance across hardware platforms like NVIDIA GPUs and Google Cloud TPUs.

The new models are available for download on the Google AI Blog.

<h2><strong>What is Generative AI?</strong></h2>

Generative AI is a type of artificial intelligence that can generate 